<a href="https://colab.research.google.com/github/DoanNgocCuong/MiniProd_NLP3_LLM1_RoutingFastResponse_FineTuningT3_2025_DoanhNghiepAnhSon_/blob/main/train/RountingTask_DataV1.2_1epoch60step_Llama3_2_(1B_and_3B)_Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [4]:
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "llama-3.1",
# )

# def formatting_prompts_func(examples):
#     convos = examples["conversations"]
#     texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#     return { "text" : texts, }
# pass

# from datasets import load_dataset
# dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

# Nhập hàm get_chat_template từ module unsloth.chat_templates
from unsloth.chat_templates import get_chat_template

# Sử dụng hàm get_chat_template để cấu hình lại tokenizer theo template chat "llama-3.1"
# Lưu ý: biến tokenizer phải được định nghĩa trước đó hoặc được truyền vào từ một nguồn khác
tokenizer = get_chat_template(
    tokenizer,                      # Biến tokenizer đã có
    chat_template="llama-3.1",      # Chọn template chat "llama-3.1" để định dạng các cuộc hội thoại
)

# Định nghĩa hàm formatting_prompts_func với tham số đầu vào là examples
def formatting_prompts_func(examples):
    # Lấy danh sách các cuộc hội thoại từ key "conversations" trong đối tượng examples
    convos = examples["conversations"]
    # Áp dụng template chat cho mỗi cuộc hội thoại trong convos, không thực hiện tokenize và không thêm generation prompt
    # Kết quả là danh sách các chuỗi văn bản đã được định dạng
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    # Trả về một dictionary với key "text" chứa danh sách các chuỗi văn bản đã định dạng
    return {"text": texts,}

# Dòng pass ở đây không có tác dụng cụ thể vì đã có return, có thể được giữ lại hoặc loại bỏ tùy ý
pass

# Nhập hàm load_dataset từ thư viện datasets để tải bộ dữ liệu
from datasets import load_dataset

# Tải bộ dữ liệu "mlabonne/FineTome-100k" với phần dữ liệu huấn luyện (split="train")
dataset = load_dataset("mlabonne/FineTome-100k", split="train")


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [5]:
# In ra thông tin tổng quan của dataset
print(dataset)

# In ra 5 mẫu dữ liệu đầu tiên trong dataset
for i in range(5):
    print(f"Mẫu dữ liệu thứ {i}: ", dataset[i])


Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})
Mẫu dữ liệu thứ 0:  {'conversations': [{'from': 'human', 'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthine

In [6]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': 'How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?'},
 {'from': 'gpt',
  'value': 'Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.'}]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
# from unsloth.chat_templates import standardize_sharegpt
# dataset = standardize_sharegpt(dataset)
# dataset = dataset.map(formatting_prompts_func, batched = True,)

# Nhập hàm standardize_sharegpt từ module unsloth.chat_templates để chuẩn hóa định dạng dữ liệu
from unsloth.chat_templates import standardize_sharegpt

# Sử dụng hàm standardize_sharegpt để chuẩn hóa dataset về định dạng sharegpt
dataset = standardize_sharegpt(dataset)

# Áp dụng hàm formatting_prompts_func cho từng mẫu trong dataset bằng cách sử dụng phương thức map
# Tham số batched=True cho phép xử lý nhiều mẫu cùng lúc, giúp tăng hiệu quả khi áp dụng hàm
dataset = dataset.map(formatting_prompts_func, batched=True,)


Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Hỏi:
```
Hàm này chỉ xử lý conversations thui chứ source, với score ko xử lý nhỉ. giờ tôi đang có 1 bộ dataset khác ý nó có conversations ko có source với score
```
=> Đúng vậy. Hàm `formatting_prompts_func` mà bạn đã định nghĩa chỉ tập trung xử lý trường `conversations`, còn các trường như `source` và `score` không được ảnh hưởng. Nếu bạn có một bộ dataset khác mà chỉ chứa trường `conversations` (không có `source` và `score`), thì hàm này vẫn hoạt động bình thường, vì nó chỉ lấy và xử lý nội dung của `conversations` để tạo ra các prompt định dạng. Bạn không cần thay đổi gì nếu bạn chỉ muốn xử lý cuộc hội thoại.

Hỏi:
```
tức là nếu nó chuẩn rồi thì sao, kiểu data của mình đã chuẩn luôn system - user - assistant rồi
```


Nếu dữ liệu của bạn đã có cấu trúc chuẩn (các tin nhắn đã có khóa "role" và "content", với giá trị của "role" là "system", "user", "assistant"), thì hàm `standardize_data_formats` vẫn sẽ thực hiện các bước kiểm tra và ánh xạ như bình thường. Cụ thể:

- Hàm sẽ duyệt qua một số mẫu từ trường `conversations` và lấy ra các khóa của từng tin nhắn.  
- Nó sẽ nhận thấy rằng có đúng 2 khóa (giả sử là `"role"` và `"content"`).  
- Sau đó, dựa vào số lượng giá trị duy nhất, nó sẽ xác định khóa nào là vai trò (role) và khóa nào là nội dung (content).  
- Tiếp theo, nó sẽ kiểm tra các giá trị của khóa vai trò có nằm trong tập các alias đã định nghĩa hay không. Nếu dữ liệu của bạn đã chuẩn thì các giá trị này đã là `"system"`, `"user"` hoặc `"assistant"` nên sẽ khớp với alias đã được mapping.  
- Cuối cùng, mỗi tin nhắn sẽ được xây dựng lại với định dạng: `{"role": <role chuẩn>, "content": <nội dung>}`. Nếu dữ liệu đã chuẩn, kết quả cuối cùng sẽ giống với dữ liệu ban đầu.

Tóm lại, nếu dữ liệu của bạn đã chuẩn, hàm này sẽ không thay đổi nội dung, mà chỉ "tái đóng gói" dữ liệu theo đúng định dạng tiêu chuẩn của Hugging Face. Bạn có thể chạy hàm này mà không lo bị lỗi, nhưng về cơ bản nó sẽ không tạo ra sự khác biệt so với dữ liệu ban đầu.

We look at how the conversations are structured for item 5:

In [8]:
dataset[5]["conversations"]

[{'content': 'How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?',
  'role': 'user'},
 {'content': 'Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.',
  'role': 'assistant'}]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [9]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|

### ✅ Tóm tắt quá trình chuẩn hóa và định dạng dữ liệu với `standardize_sharegpt` + `formatting_prompts_func`

1. **`standardize_sharegpt(dataset)`**  
   → Chuyển dữ liệu về định dạng chuẩn kiểu ShareGPT:
   - Mỗi tin nhắn có dạng `{"from": ..., "value": ...}`
   - Chuyển `from = "human"` thành `"user"`, `"gpt"` thành `"assistant"` để đồng bộ vai trò.

2. **`dataset.map(formatting_prompts_func, batched=True)`**  
   → Duyệt từng hội thoại, áp dụng **chat template** để tạo chuỗi định dạng có token đặc biệt:
   - Sinh ra trường `"text"` chứa nội dung đã được chèn các thẻ như:  
     `<|begin_of_text|>`, `<|start_header_id|>user<|end_header_id|>`, `<|eot_id|>`, v.v.

---

👉 **Mục tiêu**:  
Định dạng lại hội thoại sao cho mô hình dễ hiểu vai trò `system`, `user`, `assistant` và ngữ cảnh từng lượt nói.

📌 **Kết quả**:  
Dataset có thêm cột `"text"` → dùng để huấn luyện mô hình ngôn ngữ theo phong cách chat.

## My Dataset

In [11]:
import pandas as pd
import json
from datasets import Dataset
from unsloth.chat_templates import standardize_data_formats  # Hoặc hàm tương đương

# Bước 1: Đọc file Excel
# Giả sử file nằm tại "/content/DataFineTunev1.1_conversations_format.xlsx"
df = pd.read_excel("/content/DataFineTunev1.2_conversations_format.xlsx")

# Kiểm tra nhanh 5 dòng đầu
print("Dữ liệu gốc (5 dòng đầu):")
print(df.head())

# Bước 2: Mỗi dòng trong df đã chứa một cuộc hội thoại ở dạng chuỗi JSON.
# Ta cần chuyển cột `conversations` từ chuỗi JSON sang list[dict].

all_data = []
for idx, row in df.iterrows():
    # row["conversations"] là một chuỗi JSON,
    # ví dụ: '[{"role": "system", "content": "You are..."}, {"role": "user", "content": "Hello"}, ...]'
    conv_str = row["conversations"]

    # Kiểm tra chuỗi JSON có bị rỗng, None hoặc lỗi format không
    if not isinstance(conv_str, str):
        raise ValueError(f"Hàng thứ {idx} không phải chuỗi JSON hợp lệ: {conv_str}")

    # Giải mã chuỗi JSON thành list[dict]
    conv_list = json.loads(conv_str)

    # Tạo dictionary có key "conversations" cho mỗi hàng
    all_data.append({"conversations": conv_list})

# Bước 3: Tạo Dataset của Hugging Face từ danh sách all_data
my_dataset = Dataset.from_list(all_data)

# Kiểm tra cấu trúc phần tử đầu tiên
print("\nPhần tử đầu tiên của my_dataset trước khi chuẩn hóa:")
print(my_dataset[0])

# Bước 4: Chuẩn hóa dữ liệu (nếu cần) bằng hàm standardize_data_formats
# - Hàm này sẽ đồng nhất role = system / user / assistant theo alias
my_dataset = standardize_data_formats(my_dataset)

# - Trong quá trình format, có thể `formatting_prompts_func` (hoặc một hàm template tương tự) chèn các token đặc biệt và các thẻ đánh dấu vai trò (system/user/assistant).
# - Các thẻ này giúp mô hình (hoặc pipeline huấn luyện) phân biệt ranh giới từng vai trò trong hội thoại, cũng như hỗ trợ một số tính năng nội bộ (như “end of text”, “start header”, v.v.).
# Tham số batched=True cho phép xử lý nhiều mẫu cùng lúc, giúp tăng hiệu quả khi áp dụng hàm
my_dataset = my_dataset.map(formatting_prompts_func, batched=True,)

# Kiểm tra sau chuẩn hóa
print("\nPhần tử đầu tiên sau khi chuẩn hóa:")
print(my_dataset[0])
print(my_dataset[0]["conversations"])
print(my_dataset[0]["text"])


Dữ liệu gốc (5 dòng đầu):
                                       conversations
0  [{   "role": "system",   "content": "You are a...
1  [{   "role": "system",   "content": "You are a...
2  [{   "role": "system",   "content": "You are a...
3  [{   "role": "system",   "content": "You are a...
4  [{   "role": "system",   "content": "You are a...

Phần tử đầu tiên của my_dataset trước khi chuẩn hóa:
{'conversations': [{'content': 'You are an intelligent task classification and response generation assistant.\nYour job is to analyze user commands and generate structured JSON responses according to the correct task category.\n\nYou will be given a User Input – a natural language command from the user.\n\nYour task is to:\n1. Identify the correct tool: "todo", "todo_with_calendar", "email", "article", "thutuchanhchinh".\n- `"thutuchanhchinh"`: For legal and administrative procedure queries.\n2. Determine the appropriate action based on user intent:\n   - "todo" → [create, get_all, update, delet

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/3892 [00:00<?, ? examples/s]

Map:   0%|          | 0/3892 [00:00<?, ? examples/s]


Phần tử đầu tiên sau khi chuẩn hóa:
{'conversations': [{'content': 'You are an intelligent task classification and response generation assistant.\nYour job is to analyze user commands and generate structured JSON responses according to the correct task category.\n\nYou will be given a User Input – a natural language command from the user.\n\nYour task is to:\n1. Identify the correct tool: "todo", "todo_with_calendar", "email", "article", "thutuchanhchinh".\n- `"thutuchanhchinh"`: For legal and administrative procedure queries.\n2. Determine the appropriate action based on user intent:\n   - "todo" → [create, get_all, update, delete, complete]\n   - "todo_with_calendar" → [create, get_upcoming, get_past, get_now, update, delete, invite]\n   - "email" → [compose, get_inbox, reply, save_draft, delete, mark_important]\n   - "article" → [create, get_all, get_published, update, delete, publish, save_draft]\n   - "thutuchanhchinh" → [lookup_inf]\n\n3. Extract any relevant summary_task and ev

Nội dung hiển thị cho ta thấy rõ cách thức dữ liệu đang được “đóng gói” và “định dạng” sau khi sử dụng các hàm chuẩn hóa từ thư viện `unsloth`. Cụ thể:

1. **Có trường** `"conversations"` **lưu danh sách tin nhắn**  
   Mỗi tin nhắn là một dictionary gồm:
   - `"role"`: xác định người gửi là `system`, `user`, hay `assistant`.
   - `"content"`: nội dung văn bản thực tế.

2. **Xuất hiện trường** `"text"` **chứa nội dung đã được thêm các token đặc biệt**  
   Trong đó gồm những token như:
   - `"<|begin_of_text|>"` và `"<|eot_id|>"` để đánh dấu bắt đầu/kết thúc nội dung.  
   - `"<|start_header_id|>system<|end_header_id|>"`, `"<|start_header_id|>user<|end_header_id|>"`, `"<|start_header_id|>assistant<|end_header_id|>"` để biểu diễn vai trò.  
   - Các dòng “Cutting Knowledge Date: …” và “Today Date: …” cho thấy mô phỏng ngữ cảnh hoặc metadata về ngày tháng (nếu có).  

3. **Cấu trúc text**  
   Text này trộn cả nội dung thực tế (ví dụ “You are an intelligent …,” “Tiếp tục học tiếng anh,” …) với các đoạn đánh dấu (token đặc biệt), nhờ đó mô hình huấn luyện hoặc mô hình inference có thể phân biệt ranh giới giữa các vai trò, hoặc nhận biết các siêu thông tin khác.

4. **Nhận xét tổng quan**  
   - Dữ liệu ở dạng `"conversations"` + `"text"` này rất hữu ích để **huấn luyện mô hình** theo kịch bản hội thoại (chat-based).  
   - Phần `"text"` được “render” dựa trên template, giúp **đồng nhất** việc cung cấp ngữ cảnh, role, và nội dung cho mô hình.  
   - Nếu bạn không muốn các token đặc biệt (như `"<|begin_of_text|>"`…), bạn có thể tùy biến lại phần template.  
   - Ngược lại, nếu bạn cần mô hình hiểu được ngữ cảnh “system” / “user” / “assistant” tách biệt, thì cách tạo những token này là cần thiết.

Như vậy, toàn bộ đoạn “text” này đơn giản là **sản phẩm** của quá trình chuyển đổi và định dạng nhằm phục vụ nhu cầu huấn luyện mô hình kiểu hội thoại.

### ✅ Tóm tắt: **Token được tính từ đâu đến đâu?**

- **Token được tính từ toàn bộ nội dung trong trường `"text"`**, sau khi áp dụng template ChatML.
  
- Bao gồm tất cả:
  - `<|begin_of_text|>` – bắt đầu chuỗi
  - `<|start_header_id|>system<|end_header_id|>` + nội dung system
  - `<|start_header_id|>user<|end_header_id|>` + prompt người dùng
  - `<|start_header_id|>assistant<|end_header_id|>` + câu trả lời
  - `<|eot_id|>` – đánh dấu kết thúc mỗi phần

- 👉 Vì prompt `"system"` của bạn rất dài, bạn nên **tăng `max_seq_length`** (nếu mô hình hỗ trợ), ví dụ:
  - 1024 → có thể bị cắt
  - 2048 hoặc 4096 → an toàn hơn

- ✅ Bạn có thể dùng `tokenizer(text)` để đếm chính xác số token.

---

Nếu cần, mình có thể giúp bạn:  
- Tính số token thật sự của từng mẫu  
- Gợi ý rút gọn prompt `"system"`  
- Tối ưu hóa cho LoRA hoặc cấu hình huấn luyện phù hợp hơn.

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
# from trl import SFTTrainer
# from transformers import TrainingArguments, DataCollatorForSeq2Seq
# from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         # num_train_epochs = 1, # Set this for 1 full training run.
#         max_steps = 60,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "none", # Use this for WandB etc
#     ),
# )


# Import các hàm và lớp cần thiết
# - SFTTrainer: Lớp huấn luyện mô hình kiểu "Supervised Fine-Tuning" (điều chỉnh có giám sát)
# - TrainingArguments: Đối tượng cấu hình cho quá trình huấn luyện (ví dụ số bước, lr, v.v.)
# - DataCollatorForSeq2Seq: Công cụ gom dữ liệu theo batch, phù hợp với mô hình dạng seq2seq
# - is_bfloat16_supported: Hàm kiểm tra xem máy tính có hỗ trợ kiểu số bfloat16 không
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Khởi tạo đối tượng SFTTrainer để huấn luyện mô hình
trainer = SFTTrainer(
    model = model,                       # Mô hình (đã được nạp sẵn)
    tokenizer = tokenizer,               # Tokenizer để biến văn bản thành số và ngược lại
    train_dataset = dataset,             # Dữ liệu huấn luyện (dạng Dataset)
    dataset_text_field = "text",         # Tên cột (field) chứa chuỗi văn bản trong dataset
    max_seq_length = max_seq_length,     # Độ dài tối đa của mỗi chuỗi sau khi tokenizer
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    # DataCollatorForSeq2Seq giúp gom các mẫu theo batch, thêm mã hoá đặc biệt nếu cần

    dataset_num_proc = 2,               # Số luồng CPU để xử lý dữ liệu song song
    packing = False,                     # Có gộp (packing) các chuỗi ngắn lại hay không
    # (tắt packing có thể huấn luyện chậm hơn, nhưng code dễ theo dõi hơn)

    # Các tham số huấn luyện được đưa vào TrainingArguments
    args = TrainingArguments(
        per_device_train_batch_size = 2, # Mỗi thiết bị (GPU/CPU) sẽ lấy 2 mẫu cho mỗi batch
        gradient_accumulation_steps = 4, # Tích lũy gradient 4 lần trước khi cập nhật mô hình
        warmup_steps = 5,               # Số bước "hâm nóng" (warmup) trước khi vào giai đoạn chính
        max_steps = 60,                 # Huấn luyện tổng cộng 60 bước
        learning_rate = 2e-4,           # Tốc độ học (learning rate) là 0.0002
        fp16 = not is_bfloat16_supported(), # Nếu máy không hỗ trợ bfloat16 thì dùng fp16
        bf16 = is_bfloat16_supported(), # Nếu máy hỗ trợ bfloat16 thì dùng bfloat16
        logging_steps = 1,              # Cứ 1 bước thì in thông tin log (tiến trình) 1 lần
        optim = "adamw_8bit",           # Sử dụng trình tối ưu AdamW, tính toán với số 8-bit để tiết kiệm bộ nhớ
        weight_decay = 0.01,            # Hệ số trừng phạt độ lớn tham số (regularization)
        lr_scheduler_type = "linear",   # Kiểu thay đổi learning rate (tuyến tính)
        seed = 3407,                    # Hạt ngẫu nhiên (random seed) để kết quả lặp lại được
        output_dir = "outputs",         # Thư mục để lưu kết quả và mô hình sau huấn luyện
        report_to = "none",             # Chỗ báo cáo kết quả (none = không báo)
    ),
)


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Đây là thông báo quá trình xử lý dữ liệu khi huấn luyện, mỗi dòng cho biết một bước và tiến độ thực hiện:

1. **Converting train dataset to ChatML (num_proc=2): 100%**  
   - Hệ thống đang **chuyển đổi** bộ dữ liệu huấn luyện (train dataset) sang **định dạng ChatML** (một định dạng đặc biệt để mô hình hiểu được hội thoại).  
   - `num_proc=2` cho biết đang xử lý bằng 2 luồng (hoặc 2 tiến trình) song song.  
   - Khi báo `100000/100000 [00:24<00:00, 1501.55 examples/s]` nghĩa là đã xử lý xong **100.000 mẫu** trong **24 giây**, tốc độ khoảng **1.501 mẫu/giây**.

2. **Applying chat template to train dataset (num_proc=2): 100%**  
   - Sau bước chuyển đổi sang ChatML, hệ thống **áp dụng một “chat template”** (một khuôn mẫu định dạng) cho dữ liệu.  
   - Dữ liệu được duyệt qua và bổ sung các token đặc biệt (ví dụ: hiển thị user/assistant, cắt bớt, v.v.) theo template mà bạn đã cấu hình.  
   - Tương tự, `100000/100000 [00:30<00:00, 4935.73 examples/s]` cho thấy đã xử lý xong 100.000 mẫu trong 30 giây, tốc độ gần 4.935 mẫu/giây.

3. **Tokenizing train dataset (num_proc=2): 100%**  
   - Đây là **bước biến đổi văn bản thành các token** – nghĩa là biến câu chữ thành các chỉ số (số nguyên) để mô hình hiểu được.  
   - Với 100.000 mẫu, quá trình này mất khoảng 3 phút 31 giây (tốc độ ~503 mẫu/giây).

4. **Truncating train dataset (num_proc=2): 100%**  
   - Bước **truncating** (cắt ngắn) sẽ giới hạn độ dài mỗi mẫu (thường là theo `max_seq_length`) để tránh vượt quá mức cho phép của mô hình.  
   - Thời gian thực hiện ngắn (chỉ 2 giây) do đây là thao tác cắt bỏ phần vượt quá. Tốc độ khá cao (~44.089 mẫu/giây).

Tóm lại, các dòng log này cho thấy **bốn bước nối tiếp** trong việc chuẩn bị dữ liệu huấn luyện:  
- Chuyển đổi sang ChatML,  
- Áp dụng template hội thoại,  
- Tokenize (mã hóa văn bản),  
- Và cuối cùng là cắt ngắn dữ liệu vượt quá độ dài cho phép.  

Mỗi bước có `num_proc=2` nghĩa là chia việc xử lý cho 2 tiến trình, tăng tốc độ so với việc chạy đơn luồng.

### ✅ Tóm tắt: **Token được tính từ đâu đến đâu?**

- **Token được tính từ toàn bộ nội dung trong trường `"text"`**, sau khi áp dụng template ChatML.
  
- Bao gồm tất cả:
  - `<|begin_of_text|>` – bắt đầu chuỗi
  - `<|start_header_id|>system<|end_header_id|>` + nội dung system
  - `<|start_header_id|>user<|end_header_id|>` + prompt người dùng
  - `<|start_header_id|>assistant<|end_header_id|>` + câu trả lời
  - `<|eot_id|>` – đánh dấu kết thúc mỗi phần

- 👉 Vì prompt `"system"` của bạn rất dài, bạn nên **tăng `max_seq_length`** (nếu mô hình hỗ trợ), ví dụ:
  - 1024 → có thể bị cắt
  - 2048 hoặc 4096 → an toàn hơn

- ✅ Bạn có thể dùng `tokenizer(text)` để đếm chính xác số token.

---

Nếu cần, mình có thể giúp bạn:  
- Tính số token thật sự của từng mẫu  
- Gợi ý rút gọn prompt `"system"`  
- Tối ưu hóa cho LoRA hoặc cấu hình huấn luyện phù hợp hơn.

---

Đếm = tool: 608

In [ ]:
from transformers import AutoTokenizer

# Giả sử bạn đã có tokenizer
text = my_dataset[0]["text"]  # hoặc standardized_dataset[0]["text"]
tokens = tokenizer(text, return_tensors="pt")
print(f"Số lượng token: {len(tokens['input_ids'][0])}")


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

We can see the System and Instruction prompts are successfully masked!

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

### ✅ Tóm tắt kết quả huấn luyện từ log Unsloth

---

#### 🚀 **Cấu hình huấn luyện:**
- **Tổng số mẫu:** 100,000  
- **Số epoch:** 1  
- **Số bước huấn luyện (steps):** 60  
- **Batch size:**  
  - Mỗi thiết bị: 2  
  - `gradient_accumulation_steps = 4`  
  - Tổng batch size: `2 x 4 x 1 GPU = 8`  
- **Số tham số được huấn luyện:** 24.3 triệu / 3 tỷ (~0.81%) → đang dùng **LoRA hoặc partial fine-tuning**
- ✅ **Tối ưu VRAM:** có offload gradients thông minh.

---

#### 📉 **Diễn biến Loss qua từng bước:**
- Loss dao động từ khoảng **0.45 đến 1.3**, trung bình ~**0.8–0.9**
- Có vài bước loss cao đột biến (ví dụ bước 53: **1.3178**), nhưng phần lớn giữ ở mức ổn định.
- Một số bước có loss thấp rõ rệt (ví dụ bước 51: **0.4573**, bước 34: **0.5803**) → mô hình có thể học tốt trên các mẫu đó.

---

#### 🔍 **Đánh giá nhanh:**
- Với chỉ **60 bước training**, đây là giai đoạn **warmup hoặc thử nghiệm nhanh**, không đại diện cho training dài hạn.
- Mức loss tương đối ổn định, không tăng liên tục ⇒ **mô hình đang học được**.
- Bạn có thể:
  - ✅ Dùng checkpoint này để **inference thử**.
  - 🔁 Tăng số bước (hoặc epoch) nếu muốn huấn luyện thực sự nghiêm túc.
  - 📉 Theo dõi thêm `eval loss` (nếu có tập validation) để đánh giá overfitting.

---

Nếu bạn muốn mình:
- Vẽ biểu đồ Loss  
- Gợi ý cách đánh giá output sau huấn luyện  
- Gợi ý checkpoint saving, inference hoặc tiếp tục training  

👉 cứ nói nhé!

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)  # Bật chế độ suy luận nhanh

messages = [
    {
        "role": "system",
        "content": """
You are an intelligent task classification and response generation assistant.
Your job is to analyze user commands and generate structured JSON responses according to the correct task category.

You will be given a User Input – a natural language command from the user.

Your task is to:
1. Identify the correct tool: "todo", "todo_with_calendar", "email", "article", "thutuchanhchinh".
2. Determine the appropriate action based on user intent:
   - "todo" → [create, get_all, update, delete, complete]
   - "todo_with_calendar" → [create, get_upcoming, get_past, get_now, update, delete, invite]
   - "email" → [compose, get_inbox, reply, save_draft, delete, mark_important]
   - "article" → [create, get_all, get_published, update, delete, publish, save_draft]
   - "thutuchanhchinh" → [lookup_inf]

3. Extract any relevant summary_task and event_time.

Respond ONLY with a valid JSON object following this exact template:
{
  "tool": "<tool>",
  "action": "<action>",
  "details": {
    "summary_task": "<summary>",
    "event_time": "<time or null>"
  }
}

DO NOT explain. DO NOT include markdown. DO NOT add extra text.
"""
    },
    {
        "role": "user",
        "content": "Lấy toàn bộ danh sách công việc đã note"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)


1 prompt được GPT đề xuất để chống việc gen thừa. Ngon phết. (trong khi prompt cũ của mình bị gen thừa nhiều).

Bạn vừa đưa ra hai đoạn code sử dụng mô hình ngôn ngữ (có vẻ là LLaMA 3.1 qua Unsloth) để **sinh văn bản (generate)** dựa trên input người dùng. Cả hai đoạn đều đúng, và dùng trong các tình huống hơi khác nhau. Mình sẽ **giải thích ngắn gọn** sự khác biệt và ý nghĩa từng đoạn:

---

### ✅ Đoạn 1 – **Sinh văn bản và lấy kết quả về để xử lý**
```python
outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)

tokenizer.batch_decode(outputs)
```

- ✅ Dùng `model.generate(...)` để mô hình sinh ra token mới.
- ✅ Sau đó dùng `tokenizer.batch_decode(outputs)` để chuyển token thành chuỗi văn bản (string).
- 👉 Dạng này dùng khi bạn **muốn lấy kết quả ra để lưu, phân tích, đánh giá tự động, hoặc hiển thị sau**.

---

### ✅ Đoạn 2 – **Sinh văn bản trực tiếp ra màn hình (streaming)**
```python
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 128,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)
```

- ✅ Dùng `TextStreamer` để **hiển thị output theo thời gian thực**, giống như chatbot đang "gõ chữ".
- `skip_prompt=True`: chỉ hiển thị phần mô hình sinh ra, không in lại prompt.
- 👉 Dạng này dùng khi bạn muốn **xem kết quả ngay lập tức** (live demo, chatbot, giao diện người dùng…).

---

### 🔁 Điểm chung:
- Cả hai đoạn đều:
  - Dùng template chuẩn từ `get_chat_template`.
  - Có `add_generation_prompt=True` để mô hình biết nên bắt đầu sinh tiếp.
  - Chuyển `inputs` sang `cuda` để chạy bằng GPU.

---

### ❓Nên dùng đoạn nào?

| Mục đích                        | Nên dùng đoạn |
|-------------------------------|---------------|
| Lưu kết quả, xử lý hàng loạt   | Đoạn 1        |
| Hiển thị trực tiếp như chat   | Đoạn 2        |
| Chạy trong notebook            | Cả hai đều ổn |
| Giao diện chatbot              | Đoạn 2        |

---

Nếu bạn cần demo tương tác nhiều lượt, streaming tiếng Việt đẹp hơn, hoặc nối tiếp hội thoại thì mình có thể giúp bạn mở rộng đoạn code nhé!

# SAVE MODEL

Sau khi bạn **fine-tuning xong mô hình với Unsloth**, có **2 cách chính để lưu và inference (dự đoán)**:

---

## ✅ 1. **Lưu mô hình sau khi huấn luyện**

Unsloth sử dụng LoRA → nên chỉ cần **lưu các adapter** (không cần lưu toàn bộ mô hình gốc)

### 🔹 Lưu local:
```python
model.save_pretrained("lora_model")         # Lưu adapter LoRA
tokenizer.save_pretrained("lora_model")     # Lưu tokenizer
```

📁 Sau đó bạn sẽ có thư mục `lora_model/` chứa:
- `adapter_config.json`
- `adapter_model.bin`
- `tokenizer_config.json` + các file tokenizer khác

---

### 🔹 Hoặc đẩy lên Hugging Face Hub:
```python
model.push_to_hub("your_name/lora_model", token = "your_token")
tokenizer.push_to_hub("your_name/lora_model", token = "your_token")
```

---

## ✅ 2. **Load lại và dùng để inference (dự đoán)**

Sau khi đã lưu, bạn có thể **load lại mô hình** như sau:

```python
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model",              # đường dẫn thư mục đã lưu
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)      # Kích hoạt chế độ sinh nhanh
```

---

### ➕ Inference:

```python
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

messages = [
    {"role": "user", "content": "Lập giúp tôi thời khoá biểu học tập"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    input_ids=inputs,
    streamer=streamer,
    max_new_tokens=128,
    temperature=1.5,
    min_p=0.1
)
```

---

## ✅ 3. **(Tuỳ chọn) Lưu mô hình sang định dạng khác để dùng ở nơi khác**

### 🔹 Merge thành mô hình float16 (cho vllm, TGI, triton…)
```python
model.save_pretrained_merged("model_fp16", tokenizer, save_method = "merged_16bit")
```

### 🔹 Merge thành mô hình int4 (nhẹ, tiết kiệm RAM)
```python
model.save_pretrained_merged("model_4bit", tokenizer, save_method = "merged_4bit")
```

### 🔹 Export sang GGUF (dùng cho Ollama, llama.cpp, Jan, OpenWebUI…)
```python
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
```

---

## ✅ 4. Tổng kết: Cách chọn

| Mục đích                          | Dùng cách nào                               |
|----------------------------------|---------------------------------------------|
| Tiếp tục fine-tune               | Lưu bằng `save_pretrained()`               |
| Inference bằng Python/Colab      | Dùng `FastLanguageModel.from_pretrained()` |
| Dùng mô hình ở Ollama, WebUI     | Dùng `save_pretrained_gguf()`              |
| Đẩy lên Hugging Face             | Dùng `push_to_hub()` hoặc `push_to_hub_gguf()` |
| Triển khai ở production          | Dùng `merged_16bit` hoặc `gguf`            |

---

Nếu bạn muốn mình:
- ✅ Gợi ý format file inference mẫu
- ✅ Viết đoạn code inference cụ thể cho Hugging Face, Gradio, LangChain, Ollama
- ✅ Export sang GGUF để dùng với chatbot UI

👉 Cứ bảo mình nhé!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
